In [1]:
import os
import sys
import json
sys.path.append(os.path.join(os.getcwd().replace("model_inference", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.profiler import profiler
import matplotlib.pyplot as plt
import time
from parse_dataset import binary_dataset, NetworkDataset
from split_model import SplitModelDPU, SplitModelHost
from load_models import models
from transfer_tensors import DPUSocket

In [2]:
conf = {
    "batch_size": 16,
    "epochs": 10,
    "learning_rate": 0.0001,
    "dpu": True
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
dataset_path = os.path.join(os.getcwd().replace("model_inference", ""), "datasets")

# load label_dict
json_file = os.path.join(dataset_path, "label_index.json")
with open(json_file, 'r') as file:
    label_dict = json.load(file)

# load train, val and test datasets
train_dataset_file = os.path.join(dataset_path, "train_dataset.pt")
X_train, y_train = torch.load(train_dataset_file)

val_dataset_file = os.path.join(dataset_path, "val_dataset.pt")
X_val, y_val = torch.load(val_dataset_file)

test_dataset_file = os.path.join(dataset_path, "test_dataset.pt")
X_test, y_test = torch.load(test_dataset_file)

X_train, X_val, X_test = X_train.unsqueeze(-1), X_val.unsqueeze(-1), X_test.unsqueeze(-1)
print(X_train.shape)
print(y_train.shape)

# create train, val and test datasets
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=conf["batch_size"], shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=conf["batch_size"], shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=conf["batch_size"])

torch.Size([356334, 513, 1])
torch.Size([356334])


In [5]:
class BenchmarkDPU:
    def __init__(self, dpu_model, dpu_model_path):
        self.dpu_model = dpu_model
        self.dpu_model.load(dpu_model_path)
    
    def run(self, loader):
        """Run benchmark measuring model accuracy, cpu time, memory usage and total runtime"""
        self.dpu_model.model.eval()

        # warm-up before profiling:
        for i in range(5):
            with torch.no_grad():
                data, labels = next(iter(loader))
                pred, _ = self.dpu_model.model(data)

        with torch.profiler.profile(
            activities=[profiler.ProfilerActivity.CPU],
            record_shapes=True,
            profile_memory=True,
            with_stack=True
        ) as prof:
            data, labels = next(iter(loader))
            bin_labels = binary_dataset(labels, label_dict)
            start = time.perf_counter()
            # warm-up before profiling:
            with torch.no_grad():
                pred, logits = self.dpu_model.model(data)
                logits = logits.detach()  # break the graph here
                logits = logits.unsqueeze(1)
            end = time.perf_counter()
        
        accuracy = (pred.round() == bin_labels).float().mean()
        
        # Wall-clock time
        wall_clock_time = end - start

        # CPU compute time (µs → s)
        cpu_time_total_s = sum([e.self_cpu_time_total for e in prof.key_averages()]) / 1e6  # microseconds
        cpu_utilization = cpu_time_total_s / wall_clock_time

        # peak memory during profiling
        peak_mem = max([e.cpu_memory_usage for e in prof.key_averages()])
        peak_mem = peak_mem / 1024**2 #MB

        return 100*accuracy, cpu_utilization, peak_mem, wall_clock_time, logits, labels

In [6]:
light_dpu_path = os.path.join(os.getcwd().replace("model_inference", ""), "checkpoint", "light_dpu_split_model.pth")
light_dpu_model = models["light_dpu"]
benchmark = BenchmarkDPU(light_dpu_model, light_dpu_path)

Checkpoint loaded from /home/jorgetf/testmodel/Network-Packet-ML-Model/checkpoint/light_dpu_split_model.pth!


In [7]:
acc, cpu, mem, runtime, logits, targets = benchmark.run(test_loader)

[W1024 11:20:10.737563141 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
ERROR:2025-10-24 11:20:10 2859687:2859687 DeviceProperties.cpp:47] gpuGetDeviceCount failed with code 35


In [8]:
print(f"Benchmark (dpu): Accuracy: {acc:.2f}%, CPU Usage: {cpu:.2f} cores, Memory Usage: {mem:.2f}MB, Runtime: {runtime:.2f}s")

Benchmark (dpu): Accuracy: 69.53%, CPU Usage: 1.00 cores, Memory Usage: 24.05MB, Runtime: 0.12s


In [9]:
print(logits.shape, targets.shape)

torch.Size([16, 1, 128]) torch.Size([16])
